In [22]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import plotly.express as px
import zipfile
!pip -q install itables
from itables import init_notebook_mode
init_notebook_mode(all_interactive=True)
import itables.options as opt
opt.maxBytes = 0
opt.classes = ["display", "nowrap","compact","hover"]
opt.showIndex = False
opt.style = "max-width:6000px"
pd.set_option('display.max_colwidth', 20)

<IPython.core.display.Javascript object>

This notebook executes the analysis of one or more samples at the peptide and modfication.

For each sample, the `ionbot.twbx` result file needs to be put into a seperate folder with the folder name corresponding to a unique sample name.  

In this case we will analyse the ionbot search results of three samples and created one folder for each sample:

```
PXD000561
├── PXD000561_closed
│   └── ionbot.twbx
├── PXD000561_full
│   └── ionbot.twbx
├── PXD000561_nocorr_nort
│   └── ionbot.twbx
```

We can specify this as follows:

In [23]:
experiment = "PXD000561"
samples = ["PXD000561_closed","PXD000561_full","PXD000561_nocorr_nort"] 

In [ ]:
! wget http://genesis.ugent.be/uvpublicdata/workshop-ml-proteomics/PXD000561.zip
! unzip PXD000561.zip

Next we unpack the result files:

In [24]:
for sample in samples:
    folder = "%s/%s/"%(experiment,sample)
    archive = zipfile.ZipFile("%s/ionbot.twbx"%folder)
    for file in archive.namelist():
        if file.startswith('Data/'):
            archive.extract(file, folder)

Next we parse the result files and create one Pandas DataFrame that contains all the results:

In [25]:
tmp = []
for sample in samples:
    print(sample)
    folder = "%s/%s/Data/ionbot_result/"%(experiment,sample)
    ionbot_first = pd.read_csv("%s/ionbot.first.csv"%folder)
    ionbot_first["rank"] = ["first"]*len(ionbot_first)
    ionbot_lower = pd.read_csv("%s/ionbot.lower.csv"%folder)
    ionbot_lower["rank"] = ["lower"]*len(ionbot_lower)
    ionbot = pd.concat([ionbot_first,ionbot_lower])
    ionbot = ionbot[(ionbot["database"]=="T")&(ionbot["q-value"]<=0.01)]
    ionbot["modifications"].fillna("-",inplace=True)
    ionbot_features = pd.read_csv("%s/ionbot.features.csv"%folder)
    ionbot = ionbot.merge(ionbot_features,on="ionbot_match_id",how="left")
    ionbot["sample"] = [sample]*len(ionbot)
    tmp.append(ionbot)
ionbot = pd.concat(tmp)

PXD000561_closed
PXD000561_full
PXD000561_nocorr_nort


In [26]:
ionbot.columns

Index(['ionbot_match_id', 'spectrum_title', 'scan', 'spectrum_file',
       'precursor_mass', 'peptide_mass', 'observed_retention_time', 'charge',
       'database_peptide', 'matched_peptide', 'modifications',
       'modifications_delta', 'best_tag_rank', 'corrected_retention_time',
       'unexpected_modification', 'database', 'psm_score', 'q-value', 'PEP',
       'proteins', 'rank', 'by-count', 'all-count', 'by-explained',
       'all-explained', 'by-intensity-pattern-correlation',
       'predicted_retention_time', 'rt-pred-error', 'sample'],
      dtype='object')

In [27]:
tmp = ionbot["sample"].value_counts().reset_index(level=0)
tmp.columns = ["sample","#ids"]
fig = px.bar(tmp, y='sample', x='#ids', orientation='h')
fig.update_layout(yaxis={'categoryorder':'total ascending'})
fig.show()

In [28]:
tmp = ionbot.drop_duplicates(["sample","matched_peptide","modifications"],keep="first")
tmp = tmp["sample"].value_counts().reset_index(level=0)
tmp.columns = ["sample","#ids"]
fig = px.bar(tmp, y='sample', x='#ids', orientation='h')
fig.update_layout(yaxis={'categoryorder':'total ascending'})
fig.show()

In [29]:
tmp = ionbot.drop_duplicates(["sample","matched_peptide"],keep="first")
tmp = tmp["sample"].value_counts().reset_index(level=0)
tmp.columns = ["sample","#ids"]
fig = px.bar(tmp, y='sample', x='#ids', orientation='h')
fig.update_layout(yaxis={'categoryorder':'total ascending'})
fig.show()

In [30]:
p = {}
for sample in samples:
    p[sample] = list(ionbot[ionbot["sample"]==sample]["matched_peptide"].unique())

In [31]:
d = []
for sample1 in p:
    dd = []
    for sample2 in p:
        dd.append(len(set(p[sample1]).intersection(set(p[sample2]))))
    d.append(dd)
    
intersect = pd.DataFrame(d,columns=p,index=p)

In [32]:
intersect.style.background_gradient(cmap='Blues')

,PXD000561_closed,PXD000561_full,PXD000561_nocorr_nort
PXD000561_closed,64630,59578,54835
PXD000561_full,59578,75915,65157
PXD000561_nocorr_nort,54835,65157,68366


In [33]:
tmp = []
for sample in samples:
    folder = "%s/%s/Data/ionbot_result/"%(experiment,sample)
    proteins = pd.read_csv("%s/ionbot.coeluting.proteins.csv"%folder)
    proteins = proteins[proteins["is_shared_peptide"]==False]
    proteins.drop_duplicates("ionbot_match_id",inplace=True)
    proteins["sample"] = [sample]*len(proteins)
    tmp.append(proteins)
proteins = pd.concat(tmp)
proteins = proteins.merge(ionbot,on=["sample","ionbot_match_id"],how="inner")

In [34]:
proteins.head(200)

ionbot_match_id  is_shared_peptide        protein_group  \
0            19_5_1              False            H13_HUMAN   
1            10_8_1              False          ANXA6_HUMAN   
2            22_5_1              False           B2MG_HUMAN   
3             7_6_1              False          MATR3_HUMAN   
4           10_28_1              False           EZRI_HUMAN   
..              ...                ...                  ...   
195        4_8617_1              False          DOCK8_HUMAN   
196       17_8766_1              False           RS3A_HUMAN   
197        2_7755_1              False             TRYP_PIG   
198        0_7563_1              False  K1C10_HUMAN__sp|...   
199       13_8602_1              False          HSP7E_HUMAN   

     protein_group_q-value  protein_group_PEP      protein  \
0                    0.0         2.608590e-07    H13_HUMAN   
1                    0.0         3.715150e-10  ANXA6_HUMAN   
2                    0.0         3.443430e-07   B2MG_HUMAN   
3                    0.0         1.751380e-10  MATR3_HUMAN   
4                    0.0         6.146990e-08   EZRI_HUMAN   
..                   ...                  ...          ...   
195                  0.0         6.502390e-09  DOCK8_HUMAN   
196                  0.0         9.223290e-08   RS3A_HUMAN   
197                  0.0         7.693410e-12     TRYP_PIG   
198                  0.0         5.999990e-13  K1C10_HUMAN   
199                  0.0         1.196070e-04  HSP7E_HUMAN   

    position_in_protein uniprot_id  protein_length  protein_description  ...  \
0               124-131     P16402             221        Histone H1.3   ...   
1               450-457     P08133             673          Annexin A6   ...   
2                 33-40     P61769             119  Beta-2-microglob...  ...   
3               479-488     P43243             847            Matrin-3   ...   
4               510-517     P15311             586               Ezrin   ...   
..                  ...        ...             ...                  ...  ...   
195           1364-1371     Q8NF50            2099  Dedicator of cyt...  ...   
196             188-200     P61247             264  40S ribosomal pr...  ...   
197               58-78   TRYP_PIG             231         contaminant   ...   
198             346-363     P13645             593         contaminant   ...   
199             161-169     Q0VDF9             509  Heat shock 70 kD...  ...   

       PEP             proteins   rank by-count  all-count  by-explained  \
0    0.002  H13_HUMAN((124-1...  first        9         29         0.373   
1    0.041  ANXA6_HUMAN((450...  first       10         51         0.316   
2    0.115  B2MG_HUMAN((33-4...  first       12         42         0.268   
3    0.002  MATR3_HUMAN((479...  first       10         34         0.258   
4    0.004  EZRI_HUMAN((510-...  first       10         45         0.249   
..     ...                  ...    ...      ...        ...           ...   
195  0.008  DOCK8_HUMAN((136...  first        9         27         0.440   
196  0.000  RS3A_HUMAN((188-...  first       10         23         0.174   
197  0.000  TRYP_PIG((58-78)...  first       30         94         0.361   
198  0.000  K1C10_HUMAN((346...  first        9         13         0.501   
199  0.001  HSP7E_HUMAN((161...  first        9         29         0.255   

     all-explained  by-intensity-pattern-correlation predicted_retention_time  \
0            0.589                0.906                          1517.148       
1            0.630                0.987                           967.732       
2            0.588                0.887                          1916.716       
3            0.601                0.827                           857.875       
4            0.682                0.864                           972.544       
..             ...                  ...                               ...       
195          0.659                0.984                      

In [35]:
tmp = proteins["sample"].value_counts().reset_index(level=0)
tmp.columns = ["sample","#ids"]
fig = px.bar(tmp, y='sample', x='#ids', orientation='h')
fig.update_layout(yaxis={'categoryorder':'total ascending'})
fig.show()

In [36]:
tmp = proteins.drop_duplicates(["sample","matched_peptide","modifications"],keep="first")
tmp = tmp["sample"].value_counts().reset_index(level=0)
tmp.columns = ["sample","#ids"]
fig = px.bar(tmp, y='sample', x='#ids', orientation='h')
fig.update_layout(yaxis={'categoryorder':'total ascending'})
fig.show()

In [37]:
tmp = proteins.drop_duplicates(["sample","matched_peptide"],keep="first")
tmp = tmp["sample"].value_counts().reset_index(level=0)
tmp.columns = ["sample","#ids"]
fig = px.bar(tmp, y='sample', x='#ids', orientation='h')
fig.update_layout(yaxis={'categoryorder':'total ascending'})
fig.show()